In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

import warnings
warnings.filterwarnings('ignore')

import os
relative_path = "../__Fitting_Drivers/"
absolute_path = os.path.abspath(relative_path)

import sys
import time
import json
from tqdm import tqdm

sys.path.append(absolute_path)

In [2]:
process_data = pd.read_excel('../01_DataCollecting/Sim_result/Heat_exchanger_process_data_result_V5-short.xlsx', index_col=0)
process_data.reset_index(drop=True, inplace=True)

training_data = pd.read_csv('../01_DataCollecting/Sim_result/Surrogate_dataset_LHS_200.csv', index_col=0, encoding='ISO-8859-1')
training_data.reset_index(drop=True, inplace=True)

#direct_calibration = pd.read_excel('../03_Calibration_offline/Results/Direct_sim/Direct_Calibration_result_offline.xlsx', index_col=0)

In [3]:
from CalibrationWorkflow_driver import CalibrationWorkflow

In [7]:
y_fitt = {'PROD1E1IN': {'Weight': 1, 'SDEV':1},
  'PROD1OUT': {'Weight': 1, 'SDEV':1},
  'FEED1E1OUT': {'Weight': 1, 'SDEV':1},
  'FEED1OUT': {'Weight': 1, 'SDEV':1}}

absminfouling = 0.00025
absmaxfouling = 0.002936
# minimum hard limit az optimizernek , 'E1FOULING': (0.00025, 0.002936)
fitparamlimit = {'E1FOULING': {'min':absminfouling, 'max':absmaxfouling}, 'E2FOULING': {'min':absminfouling, 'max':absmaxfouling}}

In [8]:
# Simulation Cols mapping
cols_mapping = {'InputParams': {'FEED1MASSFLOW': 'C3', 'FEED1TEMP': 'C4','PROD1MASSFLOW': 'C5','PROD1TEMP': 'C6',
                          'E1FOULING': 'C11', 'E2FOULING': 'C15'},
               'OutputParams': {'PROD1E1IN': 'C3', 'PROD1OUT': 'C4', 'FEED1E1OUT': 'C7', 'FEED1OUT': 'C8'},
               'Outputuoms': {'PROD1E1IN': 'C', 'PROD1OUT': 'C', 'FEED1E1OUT': 'C', 'FEED1OUT': 'C'}
               }

        # always-one: If direct calibration data exists and is accurate, use that; else use validation data if accurate.
        # always-both: Use both validation and direct calibration data if available and accurate.
        # conditional: Only use direct calibration data if it exists and is accurate.
workflow_settings = {'calibration_type': 'direct', # direct, surrogate, online
                     'run_direct_for_not_accurate': False,
                     'stop_workingpoint' : None, # None 
                     'workflow_random_seed': None, # None
                     'debug_workflow': False,
                     'sim_validation_treshold' : 0.1, # This is the error inflexpoint of current y_fitt setting
                     'base_calibration_error_factor': 1,
                     'point_selection_type': 'always-one',  # always-one, always-both, conditional
                     'logging_level': 'INFO',  # DEBUG, INFO, WARNING, ERROR, CRITICAL
                     }
calibration_settings = {'y_fitt': y_fitt,
                        'fitparamlimit': fitparamlimit,
                        'particle_num': 20,
                        'optim_iterations': 100,
                        'c1' : 0.1,
                        'c2' : 0.4,
                        'w': 0.7,
                        'stopping_treshold' : 0.0001,
                        'stopping_obj' : 0.1,
                        'optim_debug' : False,
                        }

simulation_model_setting = { 'hy_filename' : '../00_Modell_database/Simulation/SampleModel_V2.hsc',
                            'cols_mapping' : cols_mapping, 
                            'resultindict' : True, 
                            }

surrogate_model_settings = {'load_path' : '../00_Modell_database/Sklearnmodel/sklearnann.joblib'}

online_learning_settings = {'batch_size': 1, 'replay_ratio': 0.01, 'retrain_scaler': False}


In [9]:
# Initialize the calibration workflow with the provided settings and models
workflow = CalibrationWorkflow(process_data, workflow_settings = workflow_settings, calibration_settings=calibration_settings)
workflow.set_simulation_model(simulation_model_setting = simulation_model_setting)
workflow.set_surrogate_model(surrogate_model_settings = surrogate_model_settings)
workflow.init_models()
workflow._init_calibration_settings()



[INFO] 14:02:22 - CalibrationWorkflow: Simulation model is initialized, and model loaded from: SampleModel_V2.hsc
[INFO] 14:02:22 - CalibrationWorkflow: Surrogate model is initialized from: ../00_Modell_database/Sklearnmodel/sklearnann.joblib


In [10]:
E1fouling_fixed = process_data.loc[0, 'E1FOULING']
E2fouling_fixed = process_data.loc[0, 'E2FOULING']

total_result = []
tt = 1
for k, item in process_data.iterrows():
  print(str(tt)+'/'+str(len(process_data)))


  input_grid = {'FEED1MASSFLOW': item['FEED1MASSFLOW'], 'FEED1TEMP':item['FEED1TEMP'], 
                    'PROD1MASSFLOW': item['PROD1MASSFLOW'], 'PROD1TEMP': item['PROD1TEMP'], 
                    'E1FOULING': E1fouling_fixed,'E2FOULING': E2fouling_fixed}

  
  y_true = {'PROD1E1IN': item['PROD1E1IN'], 
        'PROD1OUT': item['PROD1OUT'], 
        'FEED1E1OUT': item['FEED1E1OUT'],
        'FEED1OUT': item['FEED1OUT']}
  
  optorigi = {'E1FOULING': item['E1FOULING'], 'E2FOULING': item['E2FOULING']}

  pso = workflow._init_pso({0:input_grid}, y_true, init_type='direct')
  predicted = workflow.simulation_model.predict(input_grid)
  sim_cost = pso.calc_cost(predicted)


  rest = {
            'SampleID': k,
            'Target': y_true,
            'Calibration_Base_FinalIO': input_grid,
            'Validation_Base_SimulationRes': predicted,
            'Validation_Base_SimCost': sim_cost,
        }
  total_result.append(rest)
  tt += 1



1/530
2/530
3/530
4/530
5/530
6/530
7/530
8/530
9/530
10/530
11/530
12/530
13/530
14/530
15/530
16/530
17/530
18/530
19/530
20/530
21/530
22/530
23/530
24/530
25/530
26/530
27/530
28/530
29/530
30/530
31/530
32/530
33/530
34/530
35/530
36/530
37/530
38/530
39/530
40/530
41/530
42/530
43/530
44/530
45/530
46/530
47/530
48/530
49/530
50/530
51/530
52/530
53/530
54/530
55/530
56/530
57/530
58/530
59/530
60/530
61/530
62/530
63/530
64/530
65/530
66/530
67/530
68/530
69/530
70/530
71/530
72/530
73/530
74/530
75/530
76/530
77/530
78/530
79/530
80/530
81/530
82/530
83/530
84/530
85/530
86/530
87/530
88/530
89/530
90/530
91/530
92/530
93/530
94/530
95/530
96/530
97/530
98/530
99/530
100/530
101/530
102/530
103/530
104/530
105/530
106/530
107/530
108/530
109/530
110/530
111/530
112/530
113/530
114/530
115/530
116/530
117/530
118/530
119/530
120/530
121/530
122/530
123/530
124/530
125/530
126/530
127/530
128/530
129/530
130/530
131/530
132/530
133/530
134/530
135/530
136/530
137/530
138/530
139/

In [11]:
resdf = pd.DataFrame(total_result)
resdf

,SampleID,Target,Calibration_Base_FinalIO,Validation_Base_SimulationRes,Validation_Base_SimCost
0,0,"{'PROD1E1IN': 108.2722567668642, 'PROD1OUT': 9...","{'FEED1MASSFLOW': 18035.53430133957, 'FEED1TEM...","{'PROD1E1IN': 108.27225676686425, 'PROD1OUT': ...",1.248080e-32
1,1,"{'PROD1E1IN': 104.3535050140472, 'PROD1OUT': 9...","{'FEED1MASSFLOW': 17885.82338340906, 'FEED1TEM...","{'PROD1E1IN': 104.62878590233805, 'PROD1OUT': ...",6.646992e-06
2,2,"{'PROD1E1IN': 105.9350937121914, 'PROD1OUT': 9...","{'FEED1MASSFLOW': 18346.78716534058, 'FEED1TEM...","{'PROD1E1IN': 106.4905166326547, 'PROD1OUT': 9...",1.386277e-02
3,3,"{'PROD1E1IN': 106.0854462410004, 'PROD1OUT': 9...","{'FEED1MASSFLOW': 18031.97789720889, 'FEED1TEM...","{'PROD1E1IN': 106.57254117684795, 'PROD1OUT': ...",1.124932e-05
4,4,"{'PROD1E1IN': 105.0075133590793, 'PROD1OUT': 9...","{'FEED1MASSFLOW': 18402.51470121294, 'FEED1TEM...","{'PROD1E1IN': 105.06846101855984, 'PROD1OUT': ...",9.537461e-06
...,...,...,...,...,...
525,525,"{'PROD1E1IN': 116.9302158164906, 'PROD1OUT': 1...","{'FEED1MASSFLOW': 12954.7585825279, 'FEED1TEMP...","{'PROD1E1IN': 117.2798753183996, 'PROD1OUT': 1...",3.271312e-02
526,526,"{'PROD1E1IN': 116.2112946019632, 'PROD1OUT': 1...","{'FEED1MASSFLOW': 12771.85892666417, 'FEED1TEM...","{'PROD1E1IN': 116.50512397071839, 'PROD1OUT': ...",8.056312e-02
527,527,"{'PROD1E1IN': 116.4229042179647, 'PROD1OUT': 1...","{'FEED1MASSFLOW': 13325.12954876125, 'FEED1TEM...","{'PROD1E1IN': 116.70724130826727, 'PROD1OUT': ...",6.211546e-02
528,528,"{'PROD1E1IN': 116.8454717819132, 'PROD1OUT': 1...","{'FEED1MASSFLOW': 13512.72411503304, 'FEED1TEM...","{'PROD1E1IN': 116.9966989580708, 'PROD1OUT': 1...",1.155074e-01


In [ ]:
resdf.to_excel('./Results/00_No_Calibration.xlsx')